## 1648. Sell Diminishing-Valued Colored Balls
- Description:
  <blockquote>
    You have an `inventory` of different colored balls, and there is a customer that wants `orders` balls of **any** color.
     
    The customer weirdly values the colored balls. Each colored ball's value is the number of balls **of that color **you currently have in your `inventory`. For example, if you own `6` yellow balls, the customer would pay `6` for the first yellow ball. After the transaction, there are only `5` yellow balls left, so the next yellow ball is then valued at `5` (i.e., the value of the balls decreases as you sell more to the customer).
     
    You are given an integer array, `inventory`, where `inventory[i]` represents the number of balls of the `ith` color that you initially own. You are also given an integer `orders`, which represents the total number of balls that the customer wants. You can sell the balls **in any order**.
     
    Return  *the **maximum** total value that you can attain after selling* `orders` *colored balls* . As the answer may be too large, return it **modulo **`109+ 7`.
     
    **Example 1:**
    ![Image](https://assets.leetcode.com/uploads/2020/11/05/jj.gif)
     
    **Input:** inventory = [2,5], orders = 4
    **Output:** 14
    **Explanation:** Sell the 1st color 1 time (2) and the 2nd color 3 times (5 + 4 + 3).
    The maximum total value is 2 + 5 + 4 + 3 = 14.
     
    **Example 2:**
    **Input:** inventory = [3,5], orders = 6
    **Output:** 19
    **Explanation: **Sell the 1st color 2 times (3 + 2) and the 2nd color 4 times (5 + 4 + 3 + 2).
    The maximum total value is 3 + 2 + 5 + 4 + 3 + 2 = 19.
     
    **Constraints:**
     
    - `1 <= inventory.length <= 105`
    - `1 <= inventory[i] <= 109`
    - `1 <= orders <= min(sum(inventory[i]), 109)`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/sell-diminishing-valued-colored-balls/description/)

- Topics: Sorting, Math, Binary Search, Heap

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, Greedy Approach with Sorting and Arithmetic Progression
Solution description
- Time Complexity:  O(NlogN) due to sorting.
- Space Complexity: O(1) or O(N) depending on the sorting implementation.

In [ ]:
class Solution:
    def maxProfit(self, inventory: List[int], orders: int) -> int:
        inventory.sort(reverse=True)
        inventory.append(0) # Boundary to handle the last elements
        res = 0
        mod = 10**9 + 7
        # width is the number of ball colors that currently have the same maximum value
        width = 0
        
        for i in range(len(inventory) - 1):
            width += 1
            diff = inventory[i] - inventory[i+1]
            can_sell = width * diff
            # Think of the inventory as a histogram. In each iteration, you are trying to "slice" off the top section to bring the current tallest bars down to the height of the next tallest bar.
            if orders >= can_sell:
                # Sell all available balls between inventory[i] and inventory[i+1]
                high = inventory[i]
                # We want to sell balls starting from the current height (inventory[i]) down to the level just above the next height (inventory[i+1]), hence we add 1
                low = inventory[i+1] + 1
                # represents the number of distinct price levels (or "rows") you are selling in the current step of the algorithm
                num_terms = high - low + 1
                # Arithmetic Progression formula to calculate the sum of sold balls
                # The num_terms tells the formula how many integers are in the sequence you are summing up
                # We multiply this by width because for every price level (every "term"), we are selling width number of balls.
                res += width * (high + low) * num_terms // 2
                orders -= can_sell
            else:
                # Only some orders can be filled at this level
                
                # We calculate how many complete horizontal layers we can take from all columns.
                num_full_rows = orders // width
                
                remainder = orders % width
                
                high = inventory[i]
                # After selling num_full_rows layers, the lowest price level we reach is:
                low = inventory[i] - num_full_rows + 1
                res += width * (high + low) * num_full_rows // 2
                
                # Remaining balls are sold at the next lower price
                # Since the last full row was at price low, the next ball is sold at low - 1.
                res += remainder * (low - 1)
                orders = 0
                break
                
        return res % mod

In [ ]:
# Alt

class Solution:
    def maxProfit(self, inventory: List[int], orders: int) -> int:
        inventory.sort(reverse=True) # inventory high to low 
        inventory += [0]
        ans = 0
        k = 1
        for i in range(len(inventory)-1): 
            if inventory[i] > inventory[i+1]: 
                if k*(inventory[i] - inventory[i+1]) < orders: 
                    ans += k*(inventory[i] + inventory[i+1] + 1)*(inventory[i] - inventory[i+1])//2 # arithmic sum 
                    orders -= k*(inventory[i] - inventory[i+1])
                else: 
                    q, r = divmod(orders, k)
                    ans += k*(2*inventory[i] - q + 1) * q//2 + r*(inventory[i] - q)
                    return ans % 1_000_000_007
            k += 1

### Solution 2, Binary Search on the Minimum Sale Price (Threshold)

 Complexity: O(Nlog(max_val)). Since max_val is 109, log(max_val) is about 30. This means we iterate through the inventory roughly 30 times.
    No Sorting: This approach is beneficial if you are not allowed to modify the input or if the input size N is extremely large, making O(NlogN) more expensive than O(30N).
    Threshold Idea: Instead of looking at the "width" of the histogram, we are looking at a "horizontal cut" across the entire inventory at a specific price level.


In [ ]:
class Solution:
    def maxProfit(self, inventory: List[int], orders: int) -> int:
        mod = 10**9 + 7
    
        # Binary search to find the minimum price 'k' we will sell at
        low, high = 1, max(inventory)
        threshold = 0
        while low <= high:
            mid = (low + high) // 2
            # How many balls are available at price >= mid?
            count = sum(max(0, ball - mid + 1) for ball in inventory)
            if count >= orders:
                threshold = mid
                low = mid + 1
            else:
                high = mid - 1
                
        total_profit = 0
        total_balls_sold = 0
        
        for ball in inventory:
            if ball > threshold:
                # Sell all balls from this pile that are strictly more expensive than threshold
                # Prices: ball, ball-1, ..., threshold + 1
                num_sold = ball - threshold
                first = ball
                last = threshold + 1
                total_profit += (first + last) * num_sold // 2
                total_balls_sold += num_sold
                
        # Fill the remaining orders at exactly the threshold price
        remaining_orders = orders - total_balls_sold
        total_profit += remaining_orders * threshold
        
        return total_profit % mod

In [ ]:
# Alt
class Solution:
    def maxProfit(self, inventory: List[int], orders: int) -> int:
        fn = lambda x: sum(max(0, xx - x) for xx in inventory) # balls sold 
    
        # last true binary search 
        lo, hi = 0, 10**9
        while lo < hi: 
            mid = lo + hi + 1 >> 1
            if fn(mid) >= orders: lo = mid
            else: hi = mid - 1
        
        ans = sum((x + lo + 1)*(x - lo)//2 for x in inventory if x > lo)
        return (ans - (fn(lo) - orders) * (lo + 1)) % 1_000_000_007

### Solution 3, TLE, Naive Max Heap
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
class Solution:
    def maxProfit(self, inventory: List[int], orders: int) -> int:
        result = 0
        max_heap = []

        for ball in inventory:
            heapq.heappush(max_heap, (-1 * ball))

        while orders and max_heap:
            curr_highest_ball = (-1 * heapq.heappop(max_heap))

            while max_heap and curr_highest_ball >= (-1 * max_heap[0]) and orders > 0:
                result += curr_highest_ball
                orders -= 1
                curr_highest_ball -= 1
            
            heapq.heappush(max_heap, (-1 * curr_highest_ball))
    
        return result